In [ ]:
# Author: Emanuele Zeppieri <emazep@gmail.com>
#
# This code is distributed under the terms and conditions
# from the MIT License (MIT).

In [ ]:
import datetime as dt
from datetime import date, datetime, timedelta
import time
import os.path
import tabula

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
ORIGINAL_DOC_DIR = '../original_ISS_documents/bollettino_sorveglianza_integrata/'
FILE_PREFIX = 'Bollettino-sorveglianza-integrata-COVID-19_'
EXPORT_DIR = '../data/'
FILES_TO_EXCLUDE = ['Bollettino-sorveglianza-integrata-COVID-19_2020-03-09.pdf']

# Let Java wait this number of seconds in case of tabula (Java) timing issues.
WAIT = 1

In [ ]:
COLUMNS_CASES_MALE = [
    'cases_male_0-9', 'cases_male_10-19', 'cases_male_20-29',
    'cases_male_30-39', 'cases_male_40-49', 'cases_male_50-59',
    'cases_male_60-69', 'cases_male_70-79', 'cases_male_80-89',
    'cases_male_90-'
]
COLUMNS_DEATHS_MALE = [
    'deaths_male_0-9', 'deaths_male_10-19', 'deaths_male_20-29',
    'deaths_male_30-39', 'deaths_male_40-49', 'deaths_male_50-59',
    'deaths_male_60-69', 'deaths_male_70-79', 'deaths_male_80-89',
    'deaths_male_90-'
]
COLUMNS_CASES_FEMALE = [
    'cases_female_0-9', 'cases_female_10-19', 'cases_female_20-29',
    'cases_female_30-39', 'cases_female_40-49', 'cases_female_50-59',
    'cases_female_60-69', 'cases_female_70-79', 'cases_female_80-89',
    'cases_female_90-'
]
COLUMNS_DEATHS_FEMALE = [
    'deaths_female_0-9', 'deaths_female_10-19', 'deaths_female_20-29',
    'deaths_female_30-39', 'deaths_female_40-49', 'deaths_female_50-59',
    'deaths_female_60-69', 'deaths_female_70-79', 'deaths_female_80-89',
    'deaths_female_90-'
]

In [ ]:
table_pages = {
    'Bollettino-sorveglianza-integrata-COVID-19_2021-02-17.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2021-02-10.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2021-02-03.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2021-01-27.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2021-01-13.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-12-22.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-12-16.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-11-25.pdf': 23,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-10-13.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-10-06.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-29.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-22.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-15.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-08.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-09-01.pdf': 20,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-25.pdf': 21,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-18.pdf': 19,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-11.pdf': 12,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-08-04.pdf': 10,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-28.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-21.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-14.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-07-07.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-30.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-23.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-16.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-09.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-06-03.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-26.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-20.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-14.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-05-07.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-28.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-23.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-16.pdf': 7,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-09.pdf': 8,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-06.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-04-02.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-30.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-26.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-23.pdf': 6,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-16.pdf': 5,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-12.pdf': 5,
    'Bollettino-sorveglianza-integrata-COVID-19_2020-03-09.pdf': 4
}
DEFAULT_TABLE_PAGE = 22

In [ ]:
# Utility: to be used with new docs which size is unknown as needed (to fill the corresponding ETL_SPECS entry)
docs = ['Bollettino-sorveglianza-integrata-COVID-19_2021-02-24.pdf']

for file_name in docs:
    if file_name in FILES_TO_EXCLUDE:
        # Skip unusable ISS bulletins.
        continue
    
    print(file_name)
    file = ORIGINAL_DOC_DIR + file_name
    table_page = table_pages.get(file_name, DEFAULT_TABLE_PAGE)
    print(table_page)
    
    df_pdf = tabula.read_pdf(file, pages=table_page)[0]
    df_pdf_norm = pd.DataFrame()
    
    for c in df_pdf.columns:
        df_pdf_norm = pd.concat([df_pdf_norm, df_pdf[c].astype(str).str.split(expand=True)], axis=1)
        df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    # Drop spurious rows
    idx1 = df_pdf_norm[df_pdf_norm[0]=='0-9'].index.values[0]
    idx2 = df_pdf_norm[(df_pdf_norm[0]=='>90') | (df_pdf_norm[0]=='≥90')].index.values[0]
    df_pdf_norm = df_pdf_norm[idx1:idx2+1]
    df_pdf_norm.reset_index(drop=True, inplace=True)

    # Drop spurious columns
    df_pdf_norm.dropna(how='all', axis='columns', inplace=True)
    df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    print(df_pdf_norm.size)
    print(str(len(df_pdf_norm)) + '*' + str(len(df_pdf_norm.columns)))
    print()


df_pdf_norm

In [ ]:
ETL_SPECS = {
    '10*16': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*17': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*18': {
        'col_cases_male': 1, 'col_deaths_male': 3,
        'col_cases_female': 6, 'col_deaths_female': 8
    },
    '10*25': {
        'col_cases_male': 1, 'col_deaths_male': 4,
        'col_cases_female': 9, 'col_deaths_female': 12
    }
}

In [ ]:
def scraped_df_to_row(
    scraped_df,
    col_cases_male, col_deaths_male,
    col_cases_female, col_deaths_female
):
    row = scraped_df.iloc[:, col_cases_male].tolist()
    row += scraped_df.iloc[:, col_deaths_male].tolist()
    
    row += scraped_df.iloc[:, col_cases_female].tolist()
    row += scraped_df.iloc[:, col_deaths_female].tolist()
    
    return [int(str(v).replace('.', '').replace(',', '').replace('-', '0')) for v in row]

In [ ]:
# ETL loop
ds = pd.DataFrame(columns = COLUMNS_CASES_MALE + COLUMNS_DEATHS_MALE + COLUMNS_CASES_FEMALE + COLUMNS_DEATHS_FEMALE)

docs = sorted(os.listdir(ORIGINAL_DOC_DIR))

for file_name in docs:
    if (
        file_name in FILES_TO_EXCLUDE or
        not file_name.startswith(FILE_PREFIX)
    ):
        # Skip unusable ISS bulletins and spurious files.
        continue
    
    print('Scraping')
    print(file_name)
    
    file = ORIGINAL_DOC_DIR + file_name
    table_page = table_pages.get(file_name, DEFAULT_TABLE_PAGE)    
    df_pdf = tabula.read_pdf(file, pages=table_page)[0]
    df_pdf_norm = pd.DataFrame()
    
    for c in df_pdf.columns:
        df_pdf_norm = pd.concat([df_pdf_norm, df_pdf[c].astype(str).str.split(expand=True)], axis=1)
        df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    # Drop spurious rows
    idx1 = df_pdf_norm[df_pdf_norm[0]=='0-9'].index.values[0]
    idx2 = df_pdf_norm[(df_pdf_norm[0]=='>90') | (df_pdf_norm[0]=='≥90')].index.values[0]
    df_pdf_norm = df_pdf_norm[idx1:idx2+1]
    df_pdf_norm.reset_index(drop=True, inplace=True)

    # Drop spurious columns
    df_pdf_norm.dropna(how='all', axis='columns', inplace=True)
    df_pdf_norm.columns = range(len(df_pdf_norm.columns))
    
    scraped_table_dim = str(len(df_pdf_norm)) + '*' + str(len(df_pdf_norm.columns))
    print(scraped_table_dim)
    
    # Flatten the whole ISS table into a single row.
    doc_date = datetime.fromisoformat(file_name[-14:-4])
    ds.loc[doc_date] = scraped_df_to_row(df_pdf_norm, **ETL_SPECS[scraped_table_dim])

    print()
    time.sleep(WAIT)

ds.tail()

In [ ]:
# Add the totals.
ds['cases_male_total'] = ds[COLUMNS_CASES_MALE].sum(axis=1).astype(int)
ds['deaths_male_total'] = ds[COLUMNS_DEATHS_MALE].sum(axis=1).astype(int)
ds['cases_female_total'] = ds[COLUMNS_CASES_FEMALE].sum(axis=1).astype(int)
ds['deaths_female_total'] = ds[COLUMNS_DEATHS_FEMALE].sum(axis=1).astype(int)

In [ ]:
# Check the transformed table
ds.tail()

In [ ]:
# Interpolate
ds_interp_linear = ds.astype(float).asfreq(freq='D').interpolate(method='linear')
ds_interp_cubic = ds.astype(float).asfreq(freq='D').interpolate(method='cubic')

ds_interp_cubic.tail(10)

In [ ]:
# Export the transformed tables to csv files.
ds.to_csv(EXPORT_DIR + 'italy_cases_deaths_by_age_sex.csv', index_label='date')
ds_interp_linear.to_csv(EXPORT_DIR + 'italy_cases_deaths_by_age_sex_interp_linear.csv', index_label='date')
ds_interp_cubic.to_csv(EXPORT_DIR + 'italy_cases_deaths_by_age_sex_interp_cubic.csv', index_label='date')

In [ ]:
# Check roundtrip
chk_df = pd.read_csv(EXPORT_DIR+'italy_cases_deaths_by_age_sex_interp_cubic.csv', index_col=0, parse_dates=True)
chk_df